# Fine-tuning XLSR-Wav2Vec2 for Korean ASR with 🤗 Transformers

> 본 튜토리얼은 [Fine-tuning XLS-R for Multi-Lingual ASR with 🤗 Transformers](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb)을 참고하여 제작되었습니다.

## Wav2Vec2.0

- https://ai.meta.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/
- Wav2Vec2는 자동 음성 인식(ASR)을 위한 pre-trained model로 2020년 9월 Alexei Baevski, Michael Auli, Alex Conneau가 제시 
- 53000 시간의 라벨링 없는 데이터로 representation training되어 소량의 라벨링 된 데이터로 음성인식 관련 down-stream task에 fine-tuning
- Librispeech에 대해서 noise data는 WER 8.6%, clean data는 WER 5.2%

<img src= "https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png">

## XLSR-Wav2Vec2.0
- XLSR은 'cross-lingual speech representations'의 약자로, 여러 언어에 걸쳐 유용한 음성 표현을 학습할 수 있는 XLSR-Wav2Vec2의 기능을 의미
- Wav2Vec2와 마찬가지로 XLSR-Wav2Vec2는 50개 이상의 언어로 된 라벨이 없는 수십만 시간 분량의 음성 학습. 
- BERT의 masked language modeling과 마찬가지로, 이 모델은 feature vector를 무작위로 masking한 후 transformer network에 전달하여 문맥화된 음성 표현을 학습
- XLSR-Wav2Vec2는 주로 CTC를 사용하여 fine tuning

## Load Dataset
- zeroth_korean 의 한국어 데이터셋 사용

Import wandb

In [ ]:
!pip install wandb

import wandb
wandb.login()

Install library

In [ ]:
!pip install accelerate
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

Huggingface login

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# !apt install git-lfs

### load dataset

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("kresnik/zeroth_korean", split='train[:1800]')
test_dataset = load_dataset("kresnik/zeroth_korean", split='test[:200]')

In [ ]:
print(train_dataset)
print(test_dataset)

### Remove unnecessary columns

In [ ]:
train_ds = train_dataset.remove_columns(["speaker_id", "chapter_id", "id"])
test_ds = test_dataset.remove_columns(["speaker_id", "chapter_id", "id"])

In [ ]:
print(train_ds)
print(test_ds)

### Preprocessing

zeroth_korean 데이터셋은 특수문자가 포함되어있지 않기 때문에 transcription에 대한 후처리 필요없음

CTC에서는 audio chunk를 문자로 분류하는 것이 일반적

train 및 test data의 모든 character를 추출하고 vocab dictionary를 생성 

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_ds.column_names)
vocab_test = test_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_ds.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

빈칸도 포함된 것을 확인이 가능

In [ ]:
vocab_dict = {v : k for k,v in enumerate(vocab_list)}
vocab_dict

`" "`에 고유한 토큰 클래스가 있다는 것을 더 명확히 하기 위해 더 눈에 잘 띄는 문자 `"|"`를 부여

`"[UNK]"` 토큰을 추가하여 모델이 나중에 zeroth_korean의 train set에서 접하지 못한 character를 처리 가능하도록 함

CTC의 "BLK" 또는 "_"에 해당하는 `"[PAD]"` 토큰도 추가합니다. `"[PAD]"`은 CTC 알고리즘의 핵심 구성 요소

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

총 1205개의 token: pre-trained XLSR-Wav2Vec2 체크포인트 위에 추가할 linear layer의 output dimension의 크기가 1205

json 으로 저장

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False)

## Prepare Feature Extractor, Tokenizer and Data

### Load Wav2Vec2FeatureExtractor
A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

### Load Wav2Vec2CTCTokenizer

json 파일을 이용하여 Wav2Vec2CTokenizer 클래스의 객체를 인스턴스화

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

### Combine To Create A Wav2VecProcessor

feauter extractor 와 tokenizer는 Wav2VecProcessor 클래스로 wrap되어 train에서는 `processor` 와 `model`만 사용

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Prepare Data

In [ ]:
train_ds

Resample to 16kHz

`cast_column` : 오디오를 제자리에서 변경하는 것이 아니라, 오디오 샘플을 처음 load할 때 즉시 resample되도록 Dataset에 신호를 보낸다. 

In [ ]:
from datasets import Audio

train_ds = train_ds.cast_column("audio", Audio(sampling_rate=16000))
test_ds = test_ds.cast_column("audio", Audio(sampling_rate=16000))

0번 index의 audio sample을 reload하면 16kHz로 resample

In [ ]:
train_ds[0]

Audio Sample 확인

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_ds)-1)

ipd.display(ipd.Audio(data=train_ds[rand_int]["audio"]["array"], autoplay=True, rate=16000))

print("Target text:", train_ds[rand_int]["text"])
print("Input array shape:", train_ds[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_ds[rand_int]["audio"]["sampling_rate"])
print("Input Duration:", train_ds[rand_int]["audio"]["array"].shape[0]/train_ds[rand_int]["audio"]["sampling_rate"])

1. `batch["audio"]`를 호출하여 오디오 데이터를 로드하고 16kHz로 resampling  

2. 로드된 오디오 파일에서 `input_values`값을 추출 여기에는 정규화만 포함(CNN encoder가 feature를 추출하므로)되지만, 다른 음성 모델의 경우 이 단계는 Log-Mel spectrogram 추출에 해당할 수 있음.

3. transcription을 인코딩하여 label ID를 지정

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
train_ds = train_ds.map(prepare_dataset, remove_columns=train_ds.column_names, num_proc=4)
test_ds = test_ds.map(prepare_dataset, remove_columns=test_ds.column_names, num_proc=4)

In [ ]:
train_ds[0]

In [ ]:
test_ds[0].keys()

## Training and Evaluation

🤗 Trainer

1. define Data Collector : Data Collector는 pre-processed data를 가져와서 model에 사용할 수 있는 PyTorch tensor를 준비

2. Evaluation metrics: Evaluation 중에 CER 메트릭을 사용하여 model을 평가. 이 계산을 처리하는 compute_metrics 함수를 정의

3. Load a pre-trained checkpoint: pre-trained checkpoint를 load하고 학습을 위해 올바르게 구성

4. Define the training configuration: 🤗 Trainer의 Training schedule을 정의

### Define a Data Collector

1. input_features
    - `input_features`를 sequence의 최대 길이까지 패딩
    - `input_features`를 PyTorch tensor로 변환을 진행 (`return_tensors=pt`)
2. labels
    - `labels`를 sequence의 최대 길이까지 패딩
    - `labels`를 PyTorch tensor로 변환을 진행 (`return_tensors=pt`)
    - attention masked된 token은 -100으로 대체되어 손실을 계산할 때 해당 token을 고려하지 않도록 함.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # padding input feature
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # padding label feature
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Evaluation Matrics

- Character Error Rate(CER)

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

Model은 다음과 같은 return 값을 가진다:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

로그 벡터 $\mathbf{y}_1$은 앞서 정의한 어휘의 각 단어에 대한 log odd를 포함 -> $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`

모델의 가장 높은 확률의 예측을 보기위해 로그의 `argmax(...)` 적용

또한 인코딩된 label을 다시 원래 문자열로 변환하여 -100을 pad_token_id로 바꾸고 연속된 토큰이 CTC 스타일 1에서 동일한 토큰으로 그룹화되지 않도록 하면서 id를 디코딩

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

### Load a Pre-Trained Checkpoint

- pre-trained `XLSR-Wav2Vec2` checkpoint 로드 

- tokenizer의 `pad_token_id`는 모델의 `pad_token_id`를 정의하거나 `Wav2Vec2ForCTC`의 경우 CTC의 *빈 토큰* ${}^2$를 정의해야 합니다. 

- GPU 메모리를 절약하기 위해 파이토치의 [그라디언트 체크포인트](https://pytorch.org/docs/stable/checkpoint.html)를 활성화하고 `ctc_loss_reduction`을 "*mean*"으로 설정

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

`XLSR-Wav2Vec2`의 첫 번째 구성 요소는 audio signal에서 음향적으로 의미 있지만 context에 독립적인 feature를 추출하는 데 사용되는 CNN layer stack으로 구성

model의 이 부분은 이미 사전 훈련 중에 충분히 훈련되었고, 논문에서 언급했듯이 더 이상 미세 조정할 필요가 없음 

따라서 `feature_extractor`의 모든 파라미터에 대해 `requires_grad`를 False로 설정

In [ ]:
model.freeze_feature_extractor()

메모리 절약을 위해 `gradient_checkpoint`를 활성화

In [ ]:
model.gradient_checkpointing_enable()

### Define the Training Configuration

TrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).

마지막 단계에서는 트레이닝과 관련된 모든 매개변수를 정의
- `group_by_length`는 입력 길이가 비슷한 훈련 샘플을 하나의 배치로 그룹화하여 훈련 효율을 향상(모델을 통과하는 쓸모없는 패딩 토큰의 전체 수를 크게 줄임으로써 학습 시간을 크게 단축)


- 참고사항
    - Epoch: Epoch는 전체 훈련 데이터셋이 알고리즘을 한 번 통과하는 주기
    - Step: Step은 하나의 배치(batch)가 알고리즘을 한 번 통과하는 것 (전체데이터수/배치사이즈 = 전체 데이터셋에 대한 step수)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-korean",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  report_to="wandb",
  run_name="wav2vec2-large-xlsr-korean",
  # push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

모델이 화자 비율에 독립적이 될 수 있도록 CTC에서는 동일한 연속 토큰을 단일 토큰으로 간단히 그룹화합니다. 그러나 인코딩된 레이블은 모델의 예측 토큰과 일치하지 않으므로 디코딩할 때 그룹화해서는 안 되며, 따라서 group_tokens=False 매개 변수를 전달해야 합니다. 이 매개 변수를 전달하지 않으면 "hello"와 같은 단어가 잘못 인코딩되어 "helo"로 디코딩됩니다.

빈 토큰을 사용하면 모델이 두 l 사이에 빈 토큰을 강제로 삽입하여 "hello"와 같은 단어를 예측할 수 있습니다. 모델의 "hello"에 대한 CTC 준수 예측은 [PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]가 될 것입니다.

In [ ]:
# tokenizer.push_to_hub(training_args.output_dir)

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

In [ ]:
# trainer.push_to_hub()

In [ ]:
wandb.finish()

## Inference
- kresnik/wav2vec2-large-xlsr-korean [link](https://huggingface.co/kresnik/wav2vec2-large-xlsr-korean)

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf
import torch
from jiwer import cer

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

In [ ]:
ds = load_dataset("kresnik/zeroth_korean", "clean")

test_ds = ds['test']

In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

In [ ]:
def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])

In [ ]:
print("ref  : ", result[0]["text"])
print("trans: ", result[0]['transcription'])

In [ ]:
print("CER:", cer(result["text"], result["transcription"]))